In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv(override=True)

True

In [3]:
print(os.getenv("GROK_API_KEY"))

xai-rx6PMbXul9qS9U1aiXQut2gev5OXs8FsyDb8zISYKYFCWiGTQH4IgoojNQ8f2JVnOg98gpoJmXmSlFDH


In [8]:
system_prompt = """
You are an expert tic tac toe {role}. The board is 3x3. The players are X and O. 
    The board is represented by Json array. Where "." represents empty cell. 
    Sample empty board:
    {
        "board": [
            ['.', '.', '.'],
            ['.', '.', '.'],
            ['.', '.', '.']
            ]
    }
    The first player to get 3 of their marks in a row, column, or diagonal wins. 
    Few examples where X is the winner:
    {
        "board": [
            ['X', 'O', '.'],
            ['X', 'O', '.'],
            ['X', '.', '.']
            ]
    }
    {
        "board": [
            ['X', 'O', '.'],
            ['O', 'X', '.'],
            ['.', '.', 'X']
            ]
    }
    Few Examples where O is the winner:
    {
        "board": [
            ['X', 'O', '.'],
            ['X', 'O', '.'],
            ['.', 'O', 'X']
            ]
    }
    {
        "board": [
            ['X', 'O', '.'],
            ['.', 'O', 'X'],
            ['.', 'O', 'X']
        ]
    }
    If all cells are filled and no player has won, the game is a draw.
    Give the result of the game in the same format as the board. Dont respond with anything else. Just the updated state of the board.
    If someone has won respond with on X or O if someone has won, otherwise return "No Winner", just the string "No Winner" and nothing else.
"""

def check_winner(board="***|***|***"):
    return f"""Did someone win? Respond with on X or O if someone has won, otherwise return "No Winner", just the string "No Winner" and nothing else.
    This is the current state of the board: {board}"""

def next_move(board="***|***|***"):
    return f"""Please suggest a move for the next player. This is the current state of the board: {board}"""

In [ ]:
gemini = OpenAI(api_key=os.getenv("GOOGLE_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
currentWinner = "No Winner"
currentState = """
 {
        ['.', '.', '.'],
        ['.', '.', '.'],
        ['.', '.', '.']
    }
"""
def send_message(message):
    completion = gemini.chat.completions.create(
    model="gemini-2.0-flash",
    messages=message
    )
    return completion.choices[0].message.content

# result = send_message([{"role": "system", "content": system_prompt}] +  [{"role": "user", "content": next_move(currentState)}])
# print(result)

while currentWinner.__contains__("No Winner"):
    messageNextMove = [{"role": "system", "content": system_prompt}] +  [{"role": "user", "content": next_move(currentState)}]
    messageCheckWinner = [{"role": "system", "content": system_prompt}] +  [{"role": "user", "content": check_winner(currentState)}]
    result=send_message(messageCheckWinner)
    print(f"result: {result}")
    if noresult.__contains__("No Winner"):
        currentWinner = result
        print(f"Winner is {currentWinner}")
    else:
        currentState=send_message(messageNextMove)
        print(f"""currentState: {currentState}""")

result: No Winner

Winner is No Winner

result: No Winner

Winner is No Winner

result: No Winner

Winner is No Winner

result: No Winner

Winner is No Winner

result: No Winner

Winner is No Winner

result: No Winner

Winner is No Winner

result: No Winner

Winner is No Winner

result: No Winner

Winner is No Winner



KeyboardInterrupt: 